In [1]:
import glob
from shutil import copyfile
import cv2
import random
import numpy as np
import codecs, json
import math
from sklearn.externals import joblib
import dlib
from sklearn.svm import SVC
import os
from itertools import starmap
import shutil
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

C:\Users\ac\Anaconda3\envs\py36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
emocoes = ["neutra", "feliz", "triste", "surpresa", "brava"]

# imagens com final 
# 00_img.bmp = neutra
# 01_img.bmp = feliz 
# etc.

In [3]:
rootpath = r'data_org'
os.mkdir(rootpath, mode=777)
os.chdir(rootpath)
for sub_folder in emocoes:
    os.mkdir(sub_folder)    
os.chdir('..')

In [4]:
faces = glob.glob("set_train//*")

In [5]:
for face in faces: 
    for sessions in glob.glob("%s//*" %face): 
         for files in glob.glob("%s//*" %sessions): 
                if files[-10:] == "00_img.bmp":
                    file_name = files[-15:-3] + "jpg"
                    copyfile(files, 'data_org/neutra/%s' %file_name)
                if files[-10:] == "01_img.bmp":
                    file_name = files[-15:-3] + "jpg"
                    copyfile(files, 'data_org/feliz/%s' %file_name)
                if files[-10:] == "02_img.bmp":
                    file_name = files[-15:-3] + "jpg"
                    copyfile(files, 'data_org/triste/%s' %file_name)
                if files[-10:] == "03_img.bmp":
                    file_name = files[-15:-3] + "jpg"
                    copyfile(files, 'data_org/surpresa/%s' %file_name)
                if files[-10:] == "04_img.bmp":
                    file_name = files[-15:-3] + "jpg"
                    copyfile(files, 'data_org/brava/%s' %file_name)                

In [6]:
faceDet = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")
faceDet2 = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt2.xml")
faceDet3 = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt.xml")
faceDet4 = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_alt_tree.xml")

def detectar_faces(emocao):

    files = glob.glob("data_org/%s//*" %emocao)  
    filenumber = 0
    
    facefeatures = ""

    for f in files:
            frame = cv2.imread(f)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
            face2 = faceDet2.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
            face3 = faceDet3.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
            face4 = faceDet4.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

            if len(face) == 1:
                facefeatures = face
            elif len(face2) == 1:
                facefeatures == face2
            elif len(face3) == 1:
                facefeatures = face3
            elif len(face4) == 1:
                facefeatures = face4
            else:
                facefeatures = ""


            for (x, y, w, h) in facefeatures: 
                print ("face encontrada no arquivo: %s" %f)
                gray = gray[y:y+h, x:x+w]
                out = cv2.resize(gray, (350, 350))
                cv2.imwrite("data_org//%s//%s.jpg" %(emocao, filenumber), out) 

            filenumber += 1 

In [7]:
for emocao in emocoes:
    detectar_faces(emocao)

C:\Users\ac\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


face encontrada no arquivo: data_org/neutra\s002-00_img.jpg
face encontrada no arquivo: data_org/neutra\s003-00_img.jpg
face encontrada no arquivo: data_org/neutra\s004-00_img.jpg
face encontrada no arquivo: data_org/neutra\s005-00_img.jpg
face encontrada no arquivo: data_org/neutra\s006-00_img.jpg
face encontrada no arquivo: data_org/neutra\s007-00_img.jpg
face encontrada no arquivo: data_org/neutra\s008-00_img.jpg
face encontrada no arquivo: data_org/neutra\s009-00_img.jpg
face encontrada no arquivo: data_org/neutra\s010-00_img.jpg
face encontrada no arquivo: data_org/neutra\s011-00_img.jpg
face encontrada no arquivo: data_org/neutra\s012-00_img.jpg
face encontrada no arquivo: data_org/neutra\s013-00_img.jpg
face encontrada no arquivo: data_org/neutra\s014-00_img.jpg
face encontrada no arquivo: data_org/neutra\s015-00_img.jpg
face encontrada no arquivo: data_org/neutra\s016-00_img.jpg
face encontrada no arquivo: data_org/neutra\s017-00_img.jpg
face encontrada no arquivo: data_org/neu

In [8]:
clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
face_det = dlib.get_frontal_face_detector()
land_pred = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [9]:
svm_clf2 = SVC(kernel='linear', probability=True, tol=1e-3)

In [10]:
def get_images(emocao):
    m_path = 'data_org//'
    i_path = glob.glob(m_path+emocao+'//*')
    random.shuffle(i_path) 
    train_paths = i_path[:int(len(i_path)*0.90)] 
    predict_paths = i_path[-int(len(i_path)*0.10):]
    return train_paths, predict_paths

def get_landmarks(image_p):
    face_detections = face_det(image_p,1)
    for k,d in enumerate(face_detections):
        shape = land_pred(image_p,d)
        x_cords = []
        y_cords = []
        for i in range(1,68):
            x_cords.append(float(shape.part(i).x))
            y_cords.append(float(shape.part(i).y))

        xmean = np.mean(x_cords)
        ymean = np.mean(y_cords)
        x_central = [(x-xmean) for x in x_cords] 
        y_central = [(y-ymean) for y in y_cords]

        if x_cords[26] == x_cords[29]: 
            anglenose = 0
        else:
            anglenose_rad = int(math.atan((y_central[26] - y_central[29]) / (x_central[26] - x_central[29])))
            anglenose = int(math.degrees(anglenose_rad))

        if anglenose < 0:
            anglenose += 90 

        else:
            anglenose -= 90 

        landmarks_v = []
        for x,y,w,z in zip(x_central,y_central,x_cords,y_cords):
            landmarks_v.append(x) 
            landmarks_v.append(y) 

            np_mean_coor = np.asarray((ymean,xmean))
            np_coor = np.asarray((z,w))
            euclid_d = np.linalg.norm(np_coor-np_mean_coor)
            landmarks_v.append(euclid_d)

            angle_rad = (math.atan((z - ymean) / (w - xmean)))
            angle_degree = math.degrees(angle_rad)
            angle_req = int(angle_degree - anglenose)
            landmarks_v.append(angle_req)

    if len(face_detections)<1:
        landmarks_v="error"

    return  landmarks_v

def org_data():
    train_data = []
    train_labels = []
    pred_data = []
    pred_labels = []

    for emo in emocoes:
        train_p, pred_p = get_images(emo)
        for im in train_p:
            img = cv2.imread(im)
            img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            clahe_gray = clahe.apply(img_gray)
            landmarks_vec = get_landmarks(clahe_gray)
            if landmarks_vec == "error":
                pass
            else:
                train_data.append(landmarks_vec)
                train_labels.append(emocoes.index(emo))

        for im in pred_p:
            img = cv2.imread(im)
            img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            clahe_gray = clahe.apply(img_gray)
            landmarks_vec = get_landmarks(clahe_gray)
            if landmarks_vec == "error":
                pass
            else:
                pred_data.append(landmarks_vec)
                pred_labels.append(emocoes.index(emo))

    return train_data,train_labels,pred_data,pred_labels

accuracy = []

In [11]:
for i in range(1,11):
    print("Treino %d de 10" %i)
    train_data, train_labels, pred_data, pred_labels = org_data()
    np_train_data = np.array(train_data)
    np_train_labels = np.array(train_labels)
    svm_clf2.fit(np_train_data,np_train_labels)
    np_test_data = np.array(pred_data)
    final_pred = svm_clf2.score(np_test_data,pred_labels)
    print("Acurácia %f" %final_pred)
    accuracy.append(final_pred)
    print(20 * "-")

Treino 1 de 10
Acurácia 0.600000
--------------------
Treino 2 de 10
Acurácia 0.640000
--------------------
Treino 3 de 10
Acurácia 0.560000
--------------------
Treino 4 de 10
Acurácia 0.760000
--------------------
Treino 5 de 10
Acurácia 0.520000
--------------------
Treino 6 de 10
Acurácia 0.680000
--------------------
Treino 7 de 10
Acurácia 0.560000
--------------------
Treino 8 de 10
Acurácia 0.560000
--------------------
Treino 9 de 10
Acurácia 0.520000
--------------------
Treino 10 de 10
Acurácia 0.600000
--------------------


In [12]:
print("\nAcurácia Média: %f " %np.mean(accuracy))


Acurácia Média: 0.600000 


In [13]:
joblib.dump(svm_clf2,'aps_modelo.pkl')

['aps_modelo.pkl']